<a href="https://colab.research.google.com/github/MuskanTiwari12/Sentimental-Analysis-Project/blob/main/task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Task 1:Environment Setup , Data Collection & Pipeline Initialization

1>Install required libraries for Data Collection Pipeline


In [7]:
from google.colab import drive
drive.mount('/content/drive')
task2_folder = "/content/drive/MyDrive/Task1.ipynb"
!mkdir -p "{task1_folder}"  # Create folder if it doesn't exist

MessageError: Error: credential propagation was unsuccessful

In [2]:
!pip install pandas              # For data handling & saving (CSV, DataFrame)
!pip install python-dotenv       # Manage API keys securely
!pip install newsapi-python      # Fetch news articles using NewsAPI
!pip install requests            # Make HTTP requests (web scraping, APIs)
#!pip install beautifulsoup4      # Parse HTML for scraping news sites
!pip install newspaper3k         # Extract structured news content (title, text, date)
!pip install tweepy              # Collect tweets from Twitter (X) API
!pip install lxml_html_clean     #cleans and sanitizes messy HTML by removing scripts, styles, and unwanted tags, ensuring clean text for analysis.

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 66.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.4 MB/s eta 0:00:00
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13540 sha256=b10450efdf7c188df61815f294ad33f80264b96c4f99a1e172213f805dff4ed2
  Stored in directory: /root/.cache/pip/wheels/a5/91/9f/00d66475960891a64867914273fcaf78df6cb04d905b104a2a
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3341 sha256=99cab277f196ebd7fde67b4a8115e9cfee3a43b2a541241a3446a7c485c0db21
  Stored in directory: /root/.cache/pip/wheels/9f/9f/fb/364871d7426d3cdd4d293dcf7e53d97f16

   Import Required Libraries



In [3]:
import os                # For file paths & environment variables
import pandas as pd      # For storing and handling collected data
from dotenv import load_dotenv   # For loading API keys securely
from newsapi import NewsApiClient   # News API client
import requests          # For API requests & scraping
#from bs4 import BeautifulSoup  # For parsing scraped HTML
from newspaper import Article   # For extracting structured news
import tweepy            # For fetching data from Twitter (X) API

Load All API Keys from Colab Secret Manage

In [4]:
from google.colab import userdata

# Load API keys from Colab Secret Manager
news_api_key = userdata.get("NEWS_API_KEY")
twitter_bearer = userdata.get("TWITTER_BEARER")
huggingface_key = userdata.get("HUGGINGFACE_API_KEY")

This reusable function fetches news articles for a given topic using NewsAPI
and returns structured data in a pandas DataFrame with selected clean columns.

In [5]:
def fetch_news_articles(api_key: str, query: str, page_size: int = 50) -> pd.DataFrame:
    """
    Fetch news articles from NewsAPI for a given topic.

    Args:
        api_key (str): NewsAPI key stored in secret manager
        query (str): Topic to search for (e.g., "Artificial Intelligence")
        page_size (int): Number of articles per request (max 100 for free plan)

    Returns:
        pd.DataFrame: Clean DataFrame of articles
    """
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": query,
        "pageSize": page_size,
        "language": "en",
        "sortBy": "publishedAt",
        "apiKey": api_key
    }

    try:
        response = requests.get(url, params=params)
        data = response.json()

        if data.get("status") != "ok":
            raise Exception(data.get("message", "Unknown API error"))

        articles = data.get("articles", [])
        if not articles:
            return pd.DataFrame()

        # Convert to DataFrame with selected clean columns
        df = pd.DataFrame(articles)[[
            "title", "author", "source", "description", "url", "publishedAt", "content"
        ]]
        df["source"] = df["source"].apply(lambda x: x.get("name") if isinstance(x, dict) else x)
        df.rename(columns={"publishedAt": "published_at"}, inplace=True)

        return df

    except Exception as e:
        print(f" Error fetching news: {e}")
        return pd.DataFrame()


In [10]:
NEWS_API_KEY = os.environ.get("NEWS_API_KEY")
os.environ["NEWS_API_KEY"] = "50bdb7ac3c504a44923f970177f409c2"


# Example: Fetch 50 AI news articles
df_news = fetch_news_articles(NEWS_API_KEY, query="Artificial Intelligence", page_size=50)

if not df_news.empty:
    df_news.to_csv("ai_news.csv", index=False)
    print("Saved", len(df_news), "articles to ai_news.csv")
else:
    print(" No data fetched")


Saved 48 articles to ai_news.csv


In [11]:

# Save API key securely in Colab environment
os.environ["NEWS_API_KEY"] = "50bdb7ac3c504a44923f970177f409c2"


## Fetching News Articles and Performing Sentiment Analysis


In [16]:
from transformers import pipeline
import pandas as pd
import requests
import os

# Load API key
NEWS_API_KEY = os.getenv("NEWS_API_KEY")

# Use 3-class model (Positive, Negative, Neutral)
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest"
)

def fetch_and_analyze_news(query="AI", max_articles=20):
    url = f"https://newsapi.org/v2/everything?q={query}&pageSize={max_articles}&apiKey={NEWS_API_KEY}"

    try:
        response = requests.get(url)
        data = response.json()

        if response.status_code != 200 or "articles" not in data:
            print("Error fetching news:", data)
            return pd.DataFrame()

        articles = data["articles"]
        results = []

        for article in articles:
            text = article.get("content") or article.get("description") or ""
            if text:
                sentiment = sentiment_pipeline(text[:512])[0]
                label = sentiment["label"].upper()
                score = sentiment["score"]

                # Map sentiment scores
                if "NEGATIVE" in label:
                    sentiment_score = -score
                elif "POSITIVE" in label:
                    sentiment_score = score
                else:  # NEUTRAL → keep its own probability
                    sentiment_score = score
            else:
                label, sentiment_score = "N/A", 0.0

            results.append({
                "title": article.get("title"),
                "source": article.get("source", {}).get("name"),
                "publishedAt": article.get("publishedAt"),
                "content": text,
                "sentiment_label": label,
                "sentiment_score": sentiment_score
            })

        df = pd.DataFrame(results)
        df.to_csv("news_with_sentiment.csv", index=False)
        print(f"✅ Saved {len(df)} articles with sentiment to news_with_sentiment.csv")
        return df

    except Exception as e:
        print("❌ Exception occurred:", str(e))
        return pd.DataFrame()

# Example usage
df_news = fetch_and_analyze_news("Artificial Intelligence", 10)
df_news.head()


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


✅ Saved 10 articles with sentiment to news_with_sentiment.csv


,title,source,publishedAt,content,sentiment_label,sentiment_score
0,Our hottest takes on AI’s wild summer,The Verge,2025-09-12T14:02:02Z,<ul><li></li><li></li><li></li></ul>\r\nOn The...,NEUTRAL,0.809480
1,Meet the Top 10 AI-Proof Jobs That Everyone Wants,Gizmodo.com,2025-08-31T19:25:08Z,AI is rapidly scaling in the workforce and cre...,NEGATIVE,-0.499058
2,Did Nvidia Just Pop an AI Bubble? Here’s What ...,Gizmodo.com,2025-08-28T10:46:19Z,Lukewarm second quarter results from AI powerh...,POSITIVE,0.728195
3,‘Tron: Ares’ Star Says Her Character Reveals a...,Gizmodo.com,2025-09-03T15:00:22Z,Even in the opening moments of the Tron: Ares ...,NEGATIVE,-0.607060
4,Gemini for Home is Google’s biggest smart home...,The Verge,2025-08-20T16:59:47Z,<ul><li></li><li></li><li></li></ul>\r\nThe al...,NEUTRAL,0.670566
